In [1]:
# This is the method that uses the MATLAB Engine API for Python
import matlab.engine
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torchvision import  models, datasets, transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import timm
import pickle
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import numpy as np
import scipy.io as scio
from scipy.io import savemat
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
import gc

In [2]:
device = torch.device('mps') if torch.backends.mps.is_available() else 'cpu'

In [3]:
eng = matlab.engine.start_matlab()

In [4]:
algorithm_output = scio.loadmat('algorithm_output_.mat')
algorithm_output_mat = algorithm_output['algorithm_output']

In [49]:
index_selection_total = torch.arange(1,700001).to(dtype=torch.float32) 

In [50]:
index_selection_total.dtype

torch.float32

In [55]:
class CustomDataset(Dataset):
    def __init__(self, algorithm_output_mat, index_selection_total):
        # convert into PyTorch tensors and remember them
        self.algorithm_output_mat = algorithm_output_mat
        self.index_selection_total = index_selection_total
        
    def __len__(self):
        # this should return the size of the dataset
        return len(self.algorithm_output_mat)
    
    def __getitem__(self, idx):
        # this should return one sample from the dataset
        algorithm_output_mat = self.algorithm_output_mat[idx]
        index_selection_total = self.index_selection_total[idx]
        return algorithm_output_mat, index_selection_total

In [56]:
dataset = CustomDataset(algorithm_output_mat, index_selection_total)

In [58]:
# First, split the dataset into train and remaining (val + test)
train_set, remaining_set = train_test_split(dataset, test_size=0.2, random_state=42)

# Now, split the remaining set into validation and test sets
val_set, test_set = train_test_split(remaining_set, test_size=0.5, random_state=42)

In [59]:
# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_set, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size= batch_size)

In [62]:
batch_alg_out_mat, batch_index_selection = next(iter(train_loader))
print(f'shape of batch feature is {batch_alg_out_mat.shape}')
print(f'shape of batch feature is {batch_index_selection.shape}')

shape of batch feature is torch.Size([25, 70])
shape of batch feature is torch.Size([25])


In [65]:
class ModelBased(nn.Module):
    def __init__(self):
        super(ModelBased, self).__init__()

        self.linear1 = nn.Linear(140, 140)
        self.bn1 = nn.BatchNorm1d(140)
        
        self.linear2 = nn.Linear(140, 280)
        self.bn2 = nn.BatchNorm1d(280)
        
        self.linear3 = nn.Linear(280, 140)
        self.bn3 = nn.BatchNorm1d(140)
        
        self.linear4 = nn.Linear(140, 140)


    def forward(self, x):
        x = F.relu(self.bn1(self.linear1(x)))
        x = F.relu(self.bn2(self.linear2(x)))
        x = F.relu(self.bn3(self.linear3(x)))
        x = self.linear4(x)

        return x

In [66]:
model = ModelBased().to(device)

In [67]:
test_data = torch.rand([32,140]).to(device)

In [68]:
test_output = model(test_data)

In [69]:
test_output.shape

torch.Size([32, 140])

In [70]:
test_output_shape = ModelBased()(torch.rand([32,140])).dtype
test_output_shape

torch.float32

In [71]:
def complex_to_interleaved_real(complex_signal):
    real_part = complex_signal.real.to(dtype=torch.float32) 
    imag_part = complex_signal.imag.to(dtype=torch.float32) 
    interleaved_signal = torch.stack((real_part, imag_part), dim=2).reshape(complex_signal.shape[0], -1)
    return interleaved_signal

In [72]:
def interleaved_real_to_complex(interleaved_signal):
    signal_length = interleaved_signal.shape[1] // 2
    real_part = interleaved_signal[:, 0::2]  # Extract even indices
    imag_part = interleaved_signal[:, 1::2]  # Extract odd indices
    complex_signal = torch.complex(real_part, imag_part)
    return complex_signal

In [73]:
def compute_papr_complex(signal):
    # Compute |x[n]|^2: Magnitude squared of the complex signal
    power_signal = torch.abs(signal)**2
    
    # Peak power
    peak_power_signal= torch.max(power_signal, dim=1).values

    # Average power
    avg_power_signal = torch.mean(power_signal, dim=1)

    # PAPR
    papr_signal = peak_power_signal / avg_power_signal
    
    return papr_signal

In [74]:
def papr_loss(signal_going_out, signal_coming_in):
    # Compute PAPR before and after
    papr_going_out = compute_papr_complex(signal_going_out)  # Transformed signal
    papr_coming_in = compute_papr_complex(signal_coming_in)  # Original signal

    # Penalize only if PAPR after is greater than PAPR before
    papr_diff = torch.relu(papr_going_out - 0.5*papr_coming_in)
    
    return torch.mean(papr_diff)

In [84]:
def prepare_for_matlab(batch_nn_out, index_selection):
    
    batch_nn_out_real = matlab.double(batch_nn_out.real.tolist())
    batch_nn_out_imag = matlab.double(batch_nn_out.imag.tolist())

    index_selection_mat = matlab.double(index_selection.tolist())

    return batch_nn_out_real, batch_nn_out_imag, index_selection_mat 

In [85]:
a, b, c = prepare_for_matlab(batch_alg_out_mat, index_selection)

In [89]:
ser_mat = eng.calculate_ser_2(a, b, c)

Error using evalin
Unrecognized function or variable 'algorithm_input'.

Error in calculate_ser_2 (line 5)
algorithm_input = evalin('base', 'algorithm_input');
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



MatlabExecutionError: 
  File /Users/ajayiidowu/PycharmProjects/pythonProject1/Papr/calculate_ser_2.m, line 5, in calculate_ser_2
Unrecognized function or variable 'algorithm_input'.


In [34]:
def ser_loss(batch_nn_out, index_selection):

    batch_nn_out_real, batch_nn_out_imag, index_selection_mat = prepare_for_matlab(batch_nn_out, index_selection)
    ser_mat = eng.calculate_ser_2(batch_nn_out_real, batch_nn_out_imag, index_selection_mat)
    ser_torch = torch.tensor(ser_mat, dtype=torch.float32)
    ser_diff = torch.relu(ser_torch[:,2] - ser_torch[:,1])
    return torch.mean(ser_diff)

In [38]:
ser_loss(batch_alg_out_mat, index_selection)

IndexError: unable to iterate over matlab.dictionary object

In [35]:
model = ModelBased().to(device)

# Define the loss functions
loss = torch.nn.MSELoss()  # For classification

# Define an optimizer (both for the encoder and the decoder!)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

#scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.01)  # Learning rate decay scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=2)

# Variables for early stopping and best parameters
best_loss = float('inf')
patience_limit = 10


best_model = None

train_losses = []
val_losses = []

alpha = 1
beta = 1
gamma = 1

# Train the model
EPOCHS = 2
for epoch in range(EPOCHS):
    running_train_loss = 0.0
    
    model.train()
    progress_bar_train = tqdm(enumerate(train_loader), total=len(train_loader), ncols=100, leave=True)
    for index, (algorithm_output_mat, index_selection) in progress_bar_train:
        # Forward pass
        algorithm_output_mat_for_nn = complex_to_interleaved_real(algorithm_output_mat)
        algorithm_output_mat_for_nn = algorithm_output_mat_for_nn.to(device)
        
        nn_output = model(algorithm_output_mat_for_nn)
        
        # Calculate loss
        initial_loss = loss(nn_output, algorithm_output_mat_for_nn)

        nn_output_control =  interleaved_real_to_complex(nn_output)
        
        papr_diff = papr_loss(nn_output_control, algorithm_output_mat_for_nn)
        ser_diff = ser_loss(nn_output_control, index_selection)
        
        train_loss = alpha*initial_loss + beta*papr_diff + gamma*ser_diff

        # Backward pass
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # Update running loss
        running_train_loss += train_loss.item()
        avg_train_loss = running_train_loss / (index + 1)

        # Print metrics
        #progress_bar_train.set_description(f'Epoch [{epoch + 1}/{EPOCHS}] MSELos:{avg_train_loss1:.4f} MSEWeig{mse_weight:.2f} CELos:{avg_train_loss2:.4f} CEWeig{ce_weight:.2f} TrLos:{avg_train_loss:.4f} Tr.Acc: {avg_train_acc*100:.2f}%')
        progress_bar_train.set_description(f'Epoch [{epoch + 1}/{EPOCHS}] T Loss:{avg_train_loss:.4f} PAPR_dff: {papr_diff:.6f} SER_dff: {ser_diff:.4f}')
    
    #train_losses.append(avg_train_loss)
    train_losses.append(avg_train_loss)

    # Validation loop
    running_val_loss = 0.0

    
    model.eval()
    progress_bar_val = tqdm(enumerate(val_loader), total=len(val_loader), ncols=100, leave=True)
    for index, (algorithm_output_mat, index_selection) in progress_bar_val:
        
        algorithm_output_mat_for_nn = complex_to_interleaved_real(algorithm_output_mat)
        algorithm_output_mat_for_nn = algorithm_output_mat_for_nn.to(device)
        
        with torch.no_grad():
            
            nn_output = model(algorithm_output_mat_for_nn)

            # Calculate losses
            val_loss = loss(nn_output, algorithm_output_mat_for_nn)

            # Update running loss
            running_val_loss += val_loss.item()
            
            avg_val_loss = running_val_loss / (index + 1)

            nn_output_control =  interleaved_real_to_complex(nn_output)
        
            papr_diff = papr_loss(nn_output_control, algorithm_output_mat_for_nn)
            ser_diff = ser_loss(nn_output_control, index_selection)

            progress_bar_val.set_description(f'Epoch [{epoch + 1}/{EPOCHS}] V Loss:{avg_val_loss:.4f} PAPR_dff: {papr_diff:.6f} SER_dff: {ser_diff:.4f}')
    
    #val_losses.append(avg_val_loss)
    val_losses.append(avg_val_loss)
    
    scheduler.step(running_val_loss)


    # Early stopping
    if avg_val_loss < best_loss:  # Now checking for the best accuracy
        best_loss = avg_val_loss
        best_epoch = epoch + 1
        best_train_loss = avg_train_loss
        patience_ = 0
        best_weights = copy.deepcopy(model.state_dict())
        print(f"Best Validation Loss is now: {best_loss:.4f} at Epoch: {best_epoch}")
    else:
        patience_ += 1
        print(f"This is Epoch: {patience_} without improvement")
        print(f"Current Validation Loss is: {avg_val_loss:.4f} at Epoch: {epoch+1}")
        print(f"Best Validation Loss remains: {best_loss:.4f} at Epoch: {best_epoch}")
        if patience_ > patience_limit:  # Patience limit before stopping
            print("Early stopping triggered! Restoring best model weights.")
            print(f"Best Validation Loss was: {best_loss:.4f} at Epoch: {best_epoch}")
            break

best_model = model.cpu()
best_model.load_state_dict(best_weights)


 0%|                                                                      | 0/2500 [00:00<?, ?it/s]

IndexError: unable to iterate over matlab.dictionary object

In [36]:
test_losses = []
running_test_loss = 0.0


progress_bar_test = tqdm(enumerate(test_loader), total=len(test_loader), ncols=100, leave=True)
for index, (algorithm_input_mat, algorithm_output_mat, main_channels_mat, symbols_store_mat) in progress_bar_test:
        
    algorithm_output_mat_for_nn = complex_to_interleaved_real(algorithm_output_mat)
    
    with torch.no_grad():
            
        nn_output = best_model(algorithm_output_mat_for_nn)

        # Calculate losses
        test_loss = loss(nn_output, algorithm_output_mat_for_nn)

        # Update running loss
        running_test_loss += test_loss.item()
            
        avg_test_loss = running_test_loss / (index + 1)

        nn_output_control =  interleaved_real_to_complex(nn_output)
        
        papr_diff = papr_loss(nn_output_control, algorithm_output_mat_for_nn)
        ser_diff = ser_loss(algorithm_input_mat, algorithm_output_mat, nn_output_control, main_channels_mat, symbols_store_mat)

        progress_bar_test.set_description(f'Epoch [{epoch + 1}/{EPOCHS}] Te Loss:{avg_test_loss:.4f} PAPR_dff: {papr_diff:.4f} SER_dff: {ser_diff:.4f}')

        
        if index < 1:
            total_nn_out = nn_output
            total_alg_in = algorithm_input_mat
            total_alg_out = algorithm_output_mat
            total_main_channels = main_channels_mat
            total_symbols = symbols_store_mat
        else:
            total_nn_out = torch.cat([total_nn_out, nn_output], dim=0, out=None)
            total_alg_in = torch.cat([total_alg_in, algorithm_input_mat], dim=0, out=None)
            total_alg_out = torch.cat([total_alg_out, algorithm_output_mat], dim=0, out=None)
            total_main_channels = torch.cat([total_main_channels, main_channels_mat], dim=0, out=None)
            total_symbols = torch.cat([total_symbols, symbols_store_mat], dim=0, out=None)


        progress_bar_val.set_description(f'Epoch [{epoch + 1}/{EPOCHS}] Te Loss:{avg_test_loss:.4f} PAPR_dff: {papr_diff:.4f} SER_dff: {ser_diff:.4f}')
    
test_losses.append(avg_test_loss)


 0%|                                                                       | 0/313 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 4, got 2)

In [101]:
total_alg_in_real, total_alg_in_imag, total_alg_out_real, total_alg_out_imag, total_nn_out_real, total_nn_out_imag, total_main_channels_real, total_main_channels_imag, total_symbols = prepare_for_matlab(total_alg_in, total_alg_out, total_nn_out, total_main_channels, total_symbols)

In [104]:
# Save all variables in a dictionary
savemat("output_from_pytorch.mat", {
    "total_alg_in_real": total_alg_in_real,
    "total_alg_in_imag": total_alg_in_imag,
    "total_alg_out_real": total_alg_out_real,
    "total_alg_out_imag": total_alg_out_imag,
    "total_nn_out_real": total_nn_out_real,
    "total_nn_out_imag": total_nn_out_imag,
    "total_main_channels_real": total_main_channels_real,
    "total_main_channels_imag": total_main_channels_imag,
    "total_symbols": total_symbols
})